In [ ]:
import mne, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fooof import FOOOF
from fooof.plts.spectra import plot_spectrum, plot_spectrum_shading
#mne.set_log_level('error')
import warnings

# Set the current working directory to be the project main folder
os.chdir('/Users/aliciacampbell/GitHub/pyline/EEG-pyline')

import basic.arrange_data as arrange
import signal_processing.pre_process as pre_process
import signal_processing.spectral_analysis as spectr

**Locating the EEG files in folders** by define the experiment sub-folder (`exp_folder`), folder with raw EEG files (`raw_folder`), folder for exporting clean EEG files (`clean_folder`), and folder for exporting the results (`results_folder`). In `raw_folder` and `clean_folder`, there is `timepoints` folder which contains `exp_conditions` folder.

During pre-processing, all the raw EEG files are cleaned from the `raw_folder/exp_folder/timepoints[i]/exp_conditions[j]` and later saved to `clean_folder/exp_folder/timepoints[i]/exp_conditions[j]`. For analysis, the same clean files are read in and worked on until the results are exported to `results_folder/exp_folder`.

In [ ]:
### DEFINE ###
raw_folder = 'Data/Raw/'
clean_folder = 'Data/Clean/'
results_folder = 'Results/'

exp_folder = 'LEISURE_T1_T2'
timepoints = ['T1', 'T2']
exp_conditions = ['EC']

### PRE-PROCESSING

(1) The raw EEG files from `raw_folder/exp_folder` (across all timepoints and two conditions) are read in, montage is set to `biosemi32`, signals are re-referenced to `average`, and cropped to `240s` to include only the "resting" part.

(2) `0.5-30 Hz FIR filter` is designed (`zero-phase, Hamming window, order 6578`) and EOG channels are used to remove EOG-related noise with the `signal-space projections (SSP)` method.

(3) Artefact rejection is done with `Autoreject` package by removing epochs which exceed the global thereshold voltage level (`global AR`) and rest of the artefactual epochs are either removed or interpreted with `local AR`.

(4) The clean EEG signals are exported to `clean_folder/exp_folder`.

In [ ]:
### DEFINE ###
montage = 'biosemi32'
eog_channels = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8'] # EOG channels + mastoids
stimulus_channel = 'Status'
reference = 'average'
epochs_duration = 5
filter_design = dict(l_freq=0.5,h_freq=30,filter_length='auto',method='fir',
                     l_trans_bandwidth='auto',h_trans_bandwidth='auto',
                     phase='zero',fir_window='hamming',fir_design='firwin')

In [ ]:
for timepoint in timepoints:
    print('Checking files in', timepoint)
    for exp in exp_conditions:
        # Set the directory in progress and find all BDF (raw EEG) files in there
        dir_inprogress = os.path.join(clean_folder, exp_folder, timepoint,exp)
        export_dir = os.path.join(clean_folder, exp_folder, timepoint, exp)
        file_dirs, subject_names = arrange.read_files(dir_inprogress, '.fif')

In [ ]:
timepoint = 'T1'
plot_psd = True

for exp in exp_conditions:
    print('------\nWorking with files in', timepoint, exp)
    # Set the directory in progress and find all BDF (raw EEG) files in there
    dir_inprogress = os.path.join(raw_folder, exp_folder, timepoint,exp)
    export_dir = os.path.join(clean_folder, exp_folder, timepoint, exp)
    file_dirs, subject_names = arrange.read_files(dir_inprogress, '.bdf')

    for i, file in enumerate(file_dirs):
        print('\n',i,'\n---\nCleaning EEG for', subject_names[i])
        # Read in the raw EEG data
        raw = mne.io.read_raw_bdf(file, infer_types=True, eog=eog_channels,
                                  stim_channel=stimulus_channel)

        # Set the right montage (Biosemi32) and set reference as average across all channels
        raw = raw.set_montage(mne.channels.make_standard_montage(montage)).load_data()\
                .set_eeg_reference(ref_channels=reference, verbose=False)

        # Find event markers for the start and end of resting state recordings
        events = mne.find_events(raw, stim_channel=stimulus_channel, consecutive=False, output='offset')
        
        # If there is 3 events, then crop the signal by the first and last event point
        if len(events) >= 3:
            tminmax = [events[0][0]/raw.info['sfreq'], events[-1][0]/raw.info['sfreq']]
            # If there is more than 3, warn the user (as probably requires manual processing)
            if len(events) > 3:
                warnings.warn('\nMore than 3 event points found for {}\n'.format(subject_names[i]))
        # If there is 1 or 2 event points, check whether they are start or end points or similar to each
        elif len(events) == 1 or len(events) == 2:
            warnings.warn('\nOnly 1 or 2 event point(s) found for {}\n'.format(subject_names[i]))

            if events[0][0] > 100000:
                tminmax = [0, events[0][0]/raw.info['sfreq']]
            else:
                tminmax = [events[0][0]/raw.info['sfreq'], None]
        else:
            tminmax = None
            warnings.warn('\nNO event points found for {}\n'.format(subject_names[i]))

        # Use the markers to crop to EEG signal to leave only the actual resting state
        if tminmax != None:
            cropped_raw = raw.crop(tmin=tminmax[0], tmax=tminmax[1])
            print(('Event markers are following:\n{}\nStarting point: {} s\nEnding point: {} s\n'
            'Total duration: {} s').format(events, tminmax[0], tminmax[1], tminmax[1]-tminmax[0]))
            # Warn if signal length is not what it is expected for a single condition
            if (230 <= tminmax[1]-tminmax[0] <= 250) != True:
                warnings.warn('\nRaw signal length is not between 230-250s for {}\n'.format(subject_names[i]))
        else:
            cropped_raw = raw
            print('Signal NOT cropped.')
        cropped_raw = cropped_raw.drop_channels(stimulus_channel)
        
        # Filter the signal with bandpass filter and remove EOG artefacts with SSP
        filt = pre_process.filter_raw_data(cropped_raw, filter_design, line_remove=None,
                                        eog_channels=eog_channels, plot_filt=False, savefig=False, verbose=False)
        
        if plot_psd == True: mne.viz.plot_raw_psd(filt, fmin=0, fmax=30)

        # Divide the filtered signal to epochs and run Autoreject artefact rejection on the epochs
        %matplotlib inline
        epochs = pre_process.artefact_rejection(filt, subject_names[i], epo_duration=epochs_duration, verbose=False)

        # (Optional) for displaying interactive EEG plots to visually inspect the signal quality
        #%matplotlib qt
        #epochs.plot(n_channels=32,n_epochs=1)

        # Try to create a directory and save the EEG file to that directory
        try:
            os.makedirs(export_dir)
        except FileExistsError:
            pass
        try:
            mne.Epochs.save(epochs,fname='{}/{}_clean-epo.fif'.format(export_dir,subject_names[i]),
                                                                  overwrite=True)
        except FileExistsError:
            pass

### POWER SPECTRUM DENSITY: BANDPOWERS & APERIODIC ACTIVITY

Estimation of `Welch's power spectrum density (PSD)` is done for all the participants at all timepoints, both conditions, for all 32 channls and six brain regions (i.e., `Left frontal`, `Right frontal`, `Left temporal`, `Right temporal`, `Left posterior`, `Right posterior`). The PSD estimates are divided into five frequency bands - `delta (1-3.9 Hz)`, `theta (4-7.9 Hz)`, `alpha (8-12 Hz)`, `beta (12.1-30 Hz)`.

Welch's PSD is calculated for `1-30 Hz` frequency range using `2.5-second Hamming window (50% overlap)` and 3 times the window (7.5-second) zero-padding.

Secondly, the PSDs are fitted with `specparam` (`FOOOF`) model to estimate aperiodic 1/f-like component in the spectra which can be described with parameters `exponent` and `offset`. The FOOOF (specparam) algorithm (version 1.0.0) was used to parameterize neural power spectra. Settings for the algorithm were set as: `peak width limits : 1-12 Hz`; `max number of peaks : infinite`; `minimum peak height : 0.225 uV^2`; `peak threshold : 2.0 uV^2`; and `aperiodic mode : fixed`. Power spectra were parameterized across the frequency range `1-30 Hz`. The aperiodic 1/f-like fit is described with the following function, where $S$ is aperiodic component, $b$ is `offset`, $F$ is vector of frequencies, and $e$ is `exponent`:

$S=b-log(F^e)$

The results are saved as Excel spreadsheets (channel-by channel and regionally) to `results_folder/exp_folder`.

In [ ]:
### DEFINE ###
b_names = ['Delta', 'Theta', 'Alpha', 'Beta']
b_freqs = [[1, 3.9], [4, 7.9], [8, 12], [12.1, 30]]
brain_regions = {'Frontal' : ['AF3', 'F3', 'FC1', 'Fz', 'AF4', 'F4', 'FC2'],
                 'Temporal' : ['F7', 'FC5', 'T7', 'F8', 'FC6', 'T8'],
                 'Posterior' : ['CP5', 'P3', 'P7', 'Pz', 'CP6', 'P4', 'P8']}
plot_rich = True
savefig = True
psd_params = dict(method='welch', fminmax=[1, 30], window='hamming', window_duration=2.5,
                  window_overlap=0.5, zero_padding=39)
fooof_params = dict(peak_width_limits=[1,12], max_n_peaks=float("inf"), min_peak_height=0.225,
                    peak_threshold=2.0, aperiodic_mode='fixed')

# Pre-create results folders for spectral analysis data
arrange.create_results_folders(exp_folder=exp_folder, results_folder=results_folder)

sns.set_palette('muted')
sns.set_style("whitegrid")

In [ ]:
df_psd_ch_bands = pd.DataFrame()
df_fooof_regions = pd.DataFrame()
# Go through all timepoints (i.e., T1, T2, etc) AND conditions (i.e., EC, EO)
for t in range(len(timepoints)):
    for c in range(len(exp_conditions)):
        # Set the directory in progress and find all FIF (clean EEG) files in there
        dir_inprogress = os.path.join(clean_folder, exp_folder, timepoints[t], exp_conditions[c])
        file_dirs, subject_names = arrange.read_files(dir_inprogress, '_clean-epo.fif')

        for i in range(len(file_dirs)):
            # Read in the clean EEG data
            epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress, subject_names[i]),
                                                                        verbose=False)
            print('\n\n{} in progress'.format(subject_names[i]))

            # Calculate Welch's power spectrum density
            [psds,freqs] = spectr.calculate_psd(epochs, subject_names[i], **psd_params,
                                                verbose=True, plot=False)
            
            # BANDPOWERS CALCULATION
            for j in range(len(b_names)):
                # Devide the PSD to frequency band bins and calculate absolute bandpowers incl. signal quality check
                psd_ch_band_temp = spectr.bandpower_per_channel(psds, freqs, b_freqs[j], b_names[j],
                                                        subject_names[i], epochs)
                # Convert the array to dataframe and add the corresponding band, timepoint and condition
                df_psd_ch_band_temp = arrange.array_to_df(subject_names[i], epochs, psd_ch_band_temp)
                df_psd_ch_band_temp.insert(0, 'Condition', exp_conditions[c]) # FIX!
                df_psd_ch_band_temp.insert(1, 'Timepoint', timepoints[t]) # FIX!
                df_psd_ch_band_temp.insert(2, 'Measure', b_names[j])

                # Concatenate it to the dataframe including all the previous subjects
                df_psd_ch_bands = pd.concat([df_psd_ch_bands, df_psd_ch_band_temp])
            
            # APERIODIC ACTIVITY ESTIMATION
            # Average all epochs together for each channel and also for each region
            psds = psds.mean(axis=(0))
            df_psds_ch = arrange.array_to_df(subject_names[i], epochs, psds).\
                                    reset_index().drop(columns='Subject')
            df_psds_regions = arrange.df_channels_to_regions(df_psds_ch, brain_regions).\
                                        reset_index().drop(columns='Subject')

            # Go through all regions of interest
            df_fooof_regs_temp = pd.DataFrame()
            for region in df_psds_regions.columns:
                psds_temp = df_psds_regions[region].to_numpy()

                # Fit the spectrum with FOOOF (specparam)
                fm = FOOOF(**fooof_params, verbose=True)
                fm.fit(freqs, psds_temp, psd_params['fminmax'])

                # Set plot styles
                data_kwargs = {'color' : 'black', 'linewidth' : 1.4, 'label' : 'Original'}
                model_kwargs = {'color' : 'red', 'linewidth' : 1.4, 'alpha' : 0.75, 'label' : 'Full model'}
                aperiodic_kwargs = {'color' : 'blue', 'linewidth' : 1.4, 'alpha' : 0.75,
                                    'linestyle' : 'dashed', 'label' : 'Aperiodic model'}
                
                # Plot power spectrum model + aperiodic fit
                fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), dpi=100)
                plot_spectrum(fm.freqs, fm.power_spectrum,
                            ax=axs, **data_kwargs)
                plot_spectrum(fm.freqs, fm.fooofed_spectrum_,
                            ax=axs, **model_kwargs)
                plot_spectrum(fm.freqs, fm._ap_fit,
                            ax=axs, **aperiodic_kwargs)
                axs.set_xlim(psd_params['fminmax'])
                axs.grid(linewidth=0.2)
                axs.set_xlabel('Frequency (Hz)')
                axs.set_ylabel('Log-normalised power (log10[uV\u00b2/Hz])')
                axs.set_title('Spectrum model fit')
                axs.legend()

                # If true, plot all the exported variables on the plots
                if plot_rich == True:
                    axs.annotate('Error: ' + str(np.round(fm.get_params('error'), 4)) +
                                '\nR\u00b2: ' + str(np.round(fm.get_params('r_squared'), 4)),
                                (0.15, 0.16), xycoords='figure fraction', color='red', fontsize=8.5)
                    axs.annotate('Exponent: ' + str(np.round(fm.get_params('aperiodic_params','exponent'), 4)) +
                                '\nOffset: ' + str(np.round(fm.get_params('aperiodic_params','offset'), 4)),
                                (0.29, 0.16), xycoords='figure fraction', color='blue', fontsize=8.5)
                
                plt.suptitle('{} region ({})'.format(region, subject_names[i]))
                plt.tight_layout()
                if savefig == True:
                    os.makedirs('{}/{}/FOOOF/'.format(results_folder, exp_folder), exist_ok=True)
                    plt.savefig(fname='{}/{}/FOOOF/{}_{}_fooof.png'.format(results_folder, exp_folder,
                                                                           subject_names[i], region), dpi=300)
                
                # Plot the figure only if the following criterias apply (meaning the fit should be visually checked)
                if (fm.get_params('aperiodic_params','exponent') <= 0) or (fm.get_params('r_squared') < 0.6) or \
                   (fm.get_params('error') > 0.15):
                    plt.show()
                else:
                    plt.close()

                # Extract exponent and offset for the current subject in current region
                df_fooof_reg_temp = pd.DataFrame(data={region: [fm.get_params('aperiodic_params','exponent'),
                                                                fm.get_params('aperiodic_params','offset'),
                                                                fm.get_params('r_squared'),
                                                                fm.get_params('error')]},
                                                index=[subject_names[i], subject_names[i],
                                                       subject_names[i], subject_names[i]])

                # Add the region data to all regions dataframe for the current subject
                df_fooof_regs_temp = pd.concat([df_fooof_regs_temp, df_fooof_reg_temp], axis=1)
            
            # Add additional information about the condition, timepoint, and which is which measure
            df_fooof_regs_temp.insert(0, 'Condition', exp_conditions[c])
            df_fooof_regs_temp.insert(1, 'Timepoint', timepoints[t])
            df_fooof_regs_temp.insert(2, 'Measure', ['Exponent', 'Offset', 'R_2', 'Error'])

            # Merge all subjects' data together to single dataframe
            df_fooof_regions = pd.concat([df_fooof_regions, df_fooof_regs_temp])
            df_fooof_regions.index.name = 'Subject'
            
# Average the channels together for the specified regions (bandpowers)
df_psd_reg_bands = arrange.df_channels_to_regions(df_psd_ch_bands.reset_index(), brain_regions)
df_psd_reg_bands = df_psd_reg_bands.set_index(df_psd_ch_bands.index)
df_psd_reg_bands.insert(0, 'Condition', df_psd_ch_bands['Condition'])
df_psd_reg_bands.insert(1, 'Timepoint', df_psd_ch_bands['Timepoint'])
df_psd_reg_bands.insert(2, 'Measure', df_psd_ch_bands['Measure'])

# Display and export the final bandpower results
display(df_psd_ch_bands)
display(df_psd_reg_bands)
df_psd_ch_bands.to_excel('{}/{}/{}_channels_bandpowers.xlsx'.format(results_folder, exp_folder, exp_folder))
df_psd_reg_bands.to_excel('{}/{}/{}_regions_bandpowers.xlsx'.format(results_folder, exp_folder, exp_folder))

# Display and export the final aperiodic activity results
display(df_fooof_regions)
df_fooof_regions.to_excel('{}/{}/{}_regions_aperiodiccomponents.xlsx'.format(results_folder, exp_folder, exp_folder))

In [ ]:
display(df_fooof_regions)

In [ ]:
# Participants chosen visually to be removed due to either bad model fit or alpha peak detected wrongly
bad_participants = ['HBA_0007_EC_T1', 'HBA_0009_EC_T1', 'HBA_0016_EC_T1',
                    'HBA_0001_EO_T1', 'HBA_0007_EO_T1', 'HBA_0009_EO_T1',
                    'HBA_0013_EO_T1', 'HBA_0014_EO_T1', 'HBA_0025_EO_T1',
                    'HBA_0042_EO_T1', 'HBA_0058_EO_T1', 'HBA_0119_EO_T1',
                    'HBA_0121_EO_T1']

# Remove the bad participants from the results dataframe AND PSDs dataframe AND demo/att dataframe
df_fooof_regions_wo_bads = df_fooof_regions.drop(index=(bad_participants))

# Export the master dataframes that do not have the "bad" participants
df_fooof_regions_wo_bads.to_excel('{}/{}/{}_regions_aperiodiccomponents_without_bad_fits.xlsx'.format(results_folder, exp_folder, exp_folder))

In [ ]:
display(df_psd_ch_bands)
display(df_psd_reg_bands)
display(df_fooof_regions_wo_bads)

In [ ]:
# Read in aperiodic spectral analysis data for all groups and regions into one dataframe

# Get Excel files location
dir_inprogress, filename, condition = arrange.read_excel_psd(exp_folder+'/FOOOF',
                                                            results_folder,
                                                            condition_strsplit='_fooof')
df = pd.DataFrame()
for i, cond in enumerate(filename):
    print(filename[i])
    print(i)
    df_temp = pd.read_excel('{}/{}.xlsx'.format(dir_inprogress, filename[i]), index_col=0, engine='openpyxl')

    df_temp['Group'] = filename[i].split('_', 3)[0]
    df_temp['Timepoint'] = filename[i].split('_', 3)[1]
    df_temp['Region'] = filename[i].split('_', 3)[2]

    df = pd.concat([df, df_temp])

df.index = [int(x[4:8]) for x in df.index]